In [49]:
import pandas as pd

In [50]:
df = pd.read_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/df_web_experiment_cleand_steps")

In [51]:
df

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test
1,1,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:26:51,Test
2,2,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:19:22,Test
3,3,9988021,580560515_7732621733,781255054_21935453173_531117,3,2017-04-17 15:19:13,Test
4,4,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:18:04,Test
...,...,...,...,...,...,...,...
317230,417650,1574008,117364417_77840596075,528720790_71583064618_169151,1,2017-05-06 23:43:27,Test
317231,417651,2908510,814969699_90652851448,562606085_36368381773_92090,1,2017-05-10 22:57:17,Control
317232,417652,2908510,814969699_90652851448,562606085_36368381773_92090,3,2017-05-10 22:56:31,Control
317233,417653,2908510,814969699_90652851448,562606085_36368381773_92090,2,2017-05-10 22:56:23,Control


In [52]:
df.isna().sum()

Unnamed: 0      0
client_id       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
Variation       0
dtype: int64

In [53]:
# Ensure date_time is datetime
df['date_time'] = pd.to_datetime(df['date_time'])

# Find the row index of the maximum date_time per visit_id
idx = df.groupby('visit_id')['date_time'].idxmax()

# Set all last_steps to empty by default
df['last_step'] = ""

# Assign process_step as last_step only for the last row of each visit
df.loc[idx, 'last_step'] = df.loc[idx, 'process_step']

In [54]:
# Calculate the total time spent in each visit
visit_time_spans = df.groupby('visit_id')['date_time'].agg(lambda x: x.max() - x.min())

# Assign each row the time spent for its visit_id
df['time_spent'] = df['visit_id'].map(visit_time_spans).astype(str).str.split(" ").str[2]

df = df[df['last_step'] != ""]

df

,Unnamed: 0,client_id,visitor_id,visit_id,process_step,date_time,Variation,last_step,time_spent
0,0,9988021,580560515_7732621733,781255054_21935453173_531117,4,2017-04-17 15:27:07,Test,4,00:10:45
8,8,8320017,39393514_33118319366,960651974_70596002104_312201,5,2017-04-05 13:10:05,Test,5,00:01:59
13,13,4033851,762728880_76361333336,949661017_22392791362_127391,5,2017-04-05 12:29:03,Control,5,00:25:03
24,24,1982004,618934751_10082078917,132494692_73738638511_70823,5,2017-04-17 12:02:42,Test,5,00:03:00
29,29,9294070,489274391_63794562906,116640698_27328181095_188045,1,2017-04-17 11:16:28,Control,1,00:00:15
...,...,...,...,...,...,...,...,...,...
317215,417634,6426623,27047012_3402532492,685634090_42070675506_389171,5,2017-05-19 05:40:14,Control,5,00:03:01
317220,417639,433098,5505424_50616523266,773565250_67577573147_389841,5,2017-05-13 15:33:04,Control,5,00:02:17
317225,417644,7049106,710348688_12741653529,412278907_8949064401_171038,1,2017-05-20 23:07:09,Test,1,00:00:00
317226,417646,1574008,117364417_77840596075,528720790_71583064618_169151,5,2017-05-06 23:51:05,Test,5,00:07:38


In [55]:
# Create function to change type of time_spent to numeric
def convert_time_spent(df):
    df['time_spent'] = pd.to_datetime(df['time_spent'], format='%H:%M:%S').dt.time
    df['time_spent'] = df['time_spent'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)
    return df

df = convert_time_spent(df)

/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_6858/2256036968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_spent'] = pd.to_datetime(df['time_spent'], format='%H:%M:%S').dt.time
/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_6858/2256036968.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_spent'] = df['time_spent'].apply(lambda x: x.hour * 3600 + x.minute * 60 + x.second)


In [56]:
# Add column to determine if last step "5" was completed or not
df['confirmed'] = df['process_step'].apply(lambda x: True if x == 5 else False)

df = df.drop(columns=['Unnamed: 0'])
df.to_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/cleaned/df_web_experiment_timespent.csv", index=False)

/var/folders/wc/048rx86x7bqbzmjn7nr46j_80000gn/T/ipykernel_6858/2667801459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['confirmed'] = df['process_step'].apply(lambda x: True if x == 5 else False)


In [ ]:
df = pd.read_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/df_web_experiment_cleand_steps")

# Creating column for repeated step by visit_id, and creating another column to display the step name if repeated
df['repeated_step'] = df.duplicated(subset=['visitor_id', 'process_step'], keep=False).astype(int)
df['step_repeated'] = df['process_step'].where(df['repeated_step'] == 1)

# replacing null values in step_repeated with 0
df['step_repeated'] = df['step_repeated'].fillna(0).astype(int)


df.drop(columns=['Unnamed: 0'], inplace=True)
df.to_csv("/Users/anasghonaim/Library/CloudStorage/OneDrive-Personal/Ironhack/Week 5/week5_6_project/data/cleaned/df_web_experiment_steps.csv", index=False)